In [1]:
!pip install opencv-python-headless numpy matplotlib

# Import Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

# Load YOLOv3 Model

In [ ]:
weights_path = '/content/yolov3-helmet.weights'
config_path = '/content/yolov3-helmet.cfg'
names_path = '/content/helmet.names'

# Load YOLO
net = cv2.dnn.readNet(weights_path, config_path)

# Load names
with open(names_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

print("Model loaded successfully!")

# Function to load an image


In [2]:
def load_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Image not found at {img_path}")
    height, width, channels = img.shape
    return img, height, width

# Function to detect helmets

In [3]:
def detect_helmets(img):
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    return outs

# Function to draw labels

In [ ]:
def draw_labels(img, class_ids, confidences, boxes, classes):
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0) if label == 'helmet' else (0, 0, 255)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, f"{label} {confidence:.2f}", (x, y - 10), font, 1, color, 2)
    return img

# Function to get bounding boxes

In [ ]:
def get_boxes(outs, height, width, conf_threshold=0.5):
    class_ids, confidences, boxes = [], [], []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    return class_ids, confidences, boxes

In [ ]:
# Path to the new image
img_path = '/content/helmet=img.jpg'  # Correct path to the uploaded image

# Load and Process the New Image
img, height, width = load_image(img_path)
outs = detect_helmets(img)
class_ids, confidences, boxes = get_boxes(outs, height, width)
img = draw_labels(img, class_ids, confidences, boxes, classes)

# Display the Image
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()
